In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sncosmo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sncosmo_vs_snana as ss
from sedfit.utils import register_bands_for_sncosmo

In [3]:
register_bands_for_sncosmo()

In [4]:
salt2par_sncosmo = ss.sncosmo_fitres_to_df('data/salt2pars_wang2009_BV_addLOSS.dat')
salt2par_snana = ss.snana_fitres_to_df(['data/CfA3_Kyle.FITRES.TEXT','data/CSP_Kyle.FITRES.TEXT','data/CfA2_Kyle.FITRES.TEXT'])
salt2par_snana['Name_upper'] = [x.upper() for x in salt2par_snana['CID']]

In [5]:
common_list = ss.commonElements([salt2par_sncosmo.Name_upper.unique(),salt2par_snana.Name_upper.unique()])

In [6]:
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('plots/lcplots.pdf')
for name in common_list:
    name = name.upper()
    meta,lc = ss.get_photometry_single_sn(name,'data/snmeta_missingSNANA.csv','data/snlc_missingSNANA.csv')
    for s in ss.commonElements([salt2par_sncosmo.loc[salt2par_sncosmo.Name_upper==name].Survey.unique(),
                                salt2par_snana.loc[salt2par_snana.Name_upper==name].Survey.unique()]):
        data = ss.convert_lc_for_sncosmo(lc.loc[lc.Survey==s])
        psncosmo = salt2par_sncosmo.loc[(salt2par_sncosmo.Name_upper==name) & (salt2par_sncosmo.Survey==s)]
        psnana = salt2par_snana.loc[(salt2par_snana.Name_upper ==name) & (salt2par_snana.Survey==s)]
        models = []
        for p,source in zip([psncosmo,psnana],['sncosmo','snana']):
            pmap = ss.get_lcpar_map(source=source)
            models.append(ss.update_sncosmo_model(p,pmap))
        fig = ss.compare_model_sncosmo(data=data,models=models,model_label=['sncosmo','snana'],figtext=','.join([name,s]),
                                       fname=pdf, format='pdf')
pdf.close()